In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/car_price/car_price/"

/content/drive/My Drive/Colab Notebooks/matrix/car_price/car_price


In [6]:
df=pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
#factorise values
suf='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorised_values=df[feat].factorize()[0]

  if suf in feat:
    df[feat]=factorised_values
  else:
    df[feat+suf]=factorised_values

In [8]:
# factorised columns without price
cat_feats= [x for x in df.columns if suf in x]
cat_feats= [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model=DecisionTreeRegressor(max_depth=5),feats=cat_feats):
  #new X, y
  X=df[feats].values
  y=df['price_value'].values

  #crate, train and test model
  model=model
  scores=cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
#decision tree
run_model()

(-19695.13091100928, 148.72570644015792)

In [15]:
#Random Forest
run_model(RandomForestRegressor(max_depth=5,n_estimators=50, random_state=0))

(-18718.657185256638, 64.5424578125788)

In [16]:
#XGBoost
run_model(xgb.XGBRegressor(max_depth=5,n_estimators=50, seed=0, learning_rate=0.1))

[14:41:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:41:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:42:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [17]:
#finding features
m=xgb.XGBRegressor(max_depth=5,n_estimators=50, seed=0, learning_rate=0.1)
m.fit(X,y)

imp=PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp,feature_names=cat_feats)

[14:43:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
feats=['param_napęd__cat'
,'param_rok-produkcji__cat'
,'param_stan__cat'
,'param_skrzynia-biegów__cat'
,'param_faktura-vat__cat'
,'param_moc__cat'
,'param_marka-pojazdu__cat'
,'feature_kamera-cofania__cat'
,'param_typ__cat'
,'param_pojemność-skokowa__cat'
,'seller_name__cat'
,'feature_wspomaganie-kierownicy__cat'
,'param_model-pojazdu__cat'
,'param_wersja__cat'
,'param_kod-silnika__cat'
,'feature_system-start-stop__cat'
,'feature_asystent-pasa-ruchu__cat'
,'feature_czujniki-parkowania-przednie__cat'
,'feature_łopatki-zmiany-biegów__cat'
,'feature_regulowane-zawieszenie__cat']

In [20]:
len(feats)

20

In [21]:
#XGBoost top 20 features
run_model(xgb.XGBRegressor(max_depth=5,n_estimators=50, seed=0, learning_rate=0.1),feats=feats)

[14:57:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
df['param_rok-produkcji__cat']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))

In [43]:
run_model(xgb.XGBRegressor(max_depth=5,n_estimators=50, seed=0, learning_rate=0.1),feats=feats)

[15:47:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [0]:
df['param_moc__cat']=df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(x.split(' ')[0]))

In [0]:
df['param_pojemność-skokowa__cat']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(x.split('cm')[0].replace(' ', '')))

In [56]:
run_model(xgb.XGBRegressor(max_depth=5,n_estimators=50, seed=0, learning_rate=0.1),feats=feats)

[15:51:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)